![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

 [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/13.1.Finetuning_Sentence_Entity_Resolver_Model.ipynb)

# 13. Finetuning Sentence Entity Resolver Model

In [ ]:
import json
import os

from google.colab import files

license_keys = files.upload()

with open(list(license_keys.keys())[0]) as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)

# Adding license key-value pairs to environment variables
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
! pip install --upgrade -q pyspark==3.1.2 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
! pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [3]:
import json
import os
import sparknlp_jsl
import sparknlp
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SparkSession
import sys, time
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *
from sparknlp_jsl.annotator import *

from sparknlp.pretrained import ResourceDownloader
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType

params = {"spark.driver.memory":"16G",
"spark.kryoserializer.buffer.max":"2000M",
"spark.driver.maxResultSize":"2000M"}

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

print (sparknlp.version())
print (sparknlp_jsl.version())

3.3.2
3.3.2


## Load Dataset

In [4]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/AskAPatient.fold-0.train.txt

Now we will create a pandas dataframe using downloaded dataset, and then convert it to a spark dataframe.



In [5]:
import pandas as pd

cols = ["conceptId","_term","term"]

aap_tr = pd.read_csv("AskAPatient.fold-0.train.txt",sep="\t", encoding="ISO-8859-1",header=None)
aap_tr.columns = cols
aap_tr["conceptId"] = aap_tr.conceptId.apply(str)

In [6]:
aap_tr.head()

,conceptId,_term,term
0,108367008,Dislocation of joint,Dislocation of joint
1,3384011000036100,Arthrotec,Arthrotec
2,166717003,Serum creatinine raised,Serum creatinine raised
3,3877011000036101,Lipitor,Lipitor
4,402234004,Foot eczema,Foot eczema


In [7]:
aap_train_sdf = spark.createDataFrame(aap_tr).drop_duplicates()
aap_train_sdf.show()

+-----------------+--------------------+--------------------+
|        conceptId|               _term|                term|
+-----------------+--------------------+--------------------+
|        161891005|            Backache|            backache|
|        418290006|             Itching|               itchy|
|         35489007|          Depression|very serious depr...|
|         10601006|  Pain in lower limb|        pain in legs|
|        386806002|  Impaired cognition|  Impaired cognition|
|        386807006|   Memory impairment| memory difficulties|
|         21499005|Feeling agitated ...|   Severe aggitation|
|        247373008|          Ankle pain|          ANKLE PAIN|
|        262286000|    Weight increased|Big weight gain i...|
|         36349006|        Burning pain|burning in back o...|
|         24184005|Finding of increa...|blood pressure ha...|
|        308921004|Neurological symptom|Neurological symptom|
|        279043006|     Pain in buttock|     Pain in buttock|
|       

In [8]:
aap_train_sdf.printSchema()

root
 |-- conceptId: string (nullable = true)
 |-- _term: string (nullable = true)
 |-- term: string (nullable = true)



In [9]:
aap_train_sdf.count()

4382

We will limit our dataframe for a faster training.

In [10]:
aap_train_sdf = aap_train_sdf.limit(1000)

Here, we will create a pipeline for adding an embeddings column to our spark dataframe.

In [11]:
documentAssembler = DocumentAssembler()\
    .setInputCol("_term")\
    .setOutputCol("ner_chunk")

bert_embeddings = BertSentenceEmbeddings.pretrained("sbiobert_base_cased_mli", "en", "clinical/models")\
    .setInputCols(["ner_chunk"])\
    .setOutputCol("bert_embeddings")\
    .setCaseSensitive(False)

snomed_emb_pipeline = Pipeline(stages = [
    documentAssembler,
    bert_embeddings])


snomed_emb_model = snomed_emb_pipeline.fit(aap_train_sdf)

snomed_data = snomed_emb_model.transform(aap_train_sdf)

sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


Here is the new training dataframe.

In [12]:
snomed_data.show()

+-----------------+--------------------+--------------------+--------------------+--------------------+
|        conceptId|               _term|                term|           ner_chunk|     bert_embeddings|
+-----------------+--------------------+--------------------+--------------------+--------------------+
|        161891005|            Backache|            backache|[{document, 0, 7,...|[{sentence_embedd...|
|        418290006|             Itching|               itchy|[{document, 0, 6,...|[{sentence_embedd...|
|         35489007|          Depression|very serious depr...|[{document, 0, 9,...|[{sentence_embedd...|
|         10601006|  Pain in lower limb|        pain in legs|[{document, 0, 17...|[{sentence_embedd...|
|        386806002|  Impaired cognition|  Impaired cognition|[{document, 0, 17...|[{sentence_embedd...|
|        386807006|   Memory impairment| memory difficulties|[{document, 0, 16...|[{sentence_embedd...|
|         21499005|Feeling agitated ...|   Severe aggitation|[{d

Now we can train our SNOMED Sentence Entity Resolver Model by using `SentenceEntityResolverApproach` .

In [13]:
bertExtractor = SentenceEntityResolverApproach()\
  .setNeighbours(25)\
  .setThreshold(1000)\
  .setInputCols("bert_embeddings")\
  .setNormalizedCol("_term")\
  .setLabelCol("conceptId")\
  .setOutputCol('snomed_code')\
  .setDistanceFunction("EUCLIDIAN")\
  .setCaseSensitive(False)

%time snomed_model = bertExtractor.fit(snomed_data)

CPU times: user 730 ms, sys: 80.3 ms, total: 811 ms
Wall time: 2min 6s


In [14]:
# save if you will need that later
snomed_model.write().overwrite().save("sbiobertresolve_snomed_model")

Lets create a new dataset and re-train our model by using this new dataset.



In [15]:
aap_tr.head()

,conceptId,_term,term
0,108367008,Dislocation of joint,Dislocation of joint
1,3384011000036100,Arthrotec,Arthrotec
2,166717003,Serum creatinine raised,Serum creatinine raised
3,3877011000036101,Lipitor,Lipitor
4,402234004,Foot eczema,Foot eczema


We need the same columns for training.

In [16]:
new_dataset = pd.DataFrame(columns=aap_tr.columns)
new_dataset

,conceptId,_term,term


## Use Cases

Now we can add our concept_codes and terms to this new dataframe. In this example we wanted to show you some cases that you can see the success of re-training feature and used `XXXX` in the codes to get easily;

- Added new lines that they are close to version of them in the main dataset (`Dislocation of joint -> Dislocation New Term`).

- Changed the code of the terms in the main dataset (`Arthrotec`)

- Changed the positions of the words in the terms (`Serum creatinine raised -> Raised serum creatinine`)

- Added new terms to the dataset (`New Drug`, `athlete's foot`)

In [17]:
new_dataset.conceptId = ["XXXX67008", "XXXXXXXX00036100", "XXXXX7003", "XXXXXXXX00036101", "XXXXX4004"]
new_dataset._term = ["Dislocation New Term", "Arthrotec", "Raised serum creatinine", "New Drug", "athlete's foot"]
new_dataset.term = ["Dislocation New Term", "Arthrotec", "Raised serum creatinine", "New Drug", "athlete's foot"]
new_dataset

,conceptId,_term,term
0,XXXX67008,Dislocation New Term,Dislocation New Term
1,XXXXXXXX00036100,Arthrotec,Arthrotec
2,XXXXX7003,Raised serum creatinine,Raised serum creatinine
3,XXXXXXXX00036101,New Drug,New Drug
4,XXXXX4004,athlete's foot,athlete's foot


We transformed our new dataframe by using `snomed_emb_model` and added new columns that we need for re-training.

In [18]:
new_snomed_data = snomed_emb_model.transform(spark.createDataFrame(new_dataset))
new_snomed_data.show()

+----------------+--------------------+--------------------+--------------------+--------------------+
|       conceptId|               _term|                term|           ner_chunk|     bert_embeddings|
+----------------+--------------------+--------------------+--------------------+--------------------+
|       XXXX67008|Dislocation New Term|Dislocation New Term|[{document, 0, 19...|[{sentence_embedd...|
|XXXXXXXX00036100|           Arthrotec|           Arthrotec|[{document, 0, 8,...|[{sentence_embedd...|
|       XXXXX7003|Raised serum crea...|Raised serum crea...|[{document, 0, 22...|[{sentence_embedd...|
|XXXXXXXX00036101|            New Drug|            New Drug|[{document, 0, 7,...|[{sentence_embedd...|
|       XXXXX4004|      athlete's foot|      athlete's foot|[{document, 0, 13...|[{sentence_embedd...|
+----------------+--------------------+--------------------+--------------------+--------------------+



Now we will re-train our main model with new dataset by using `.setPretrainedModelPath()` parameter. 

In [19]:
new_snomed_model = bertExtractor.setPretrainedModelPath("sbiobertresolve_snomed_model").fit(new_snomed_data)

In [20]:
# save if you need later

new_snomed_model.write().overwrite().save("new_sbiobertresolve_snomed_model")

Write a function to show the results more clearly.

In [21]:
import pandas as pd

pd.set_option('display.max_colwidth', 0)


def get_codes (lp, text, vocab='snomed_code'):
    
    full_light_result = lp.fullAnnotate(text)

    chunks = []
    codes = []
    begin = []
    end = []
    resolutions=[]
    all_distances =[]
    all_codes=[]
    all_cosines = []

    for chunk, code in zip(full_light_result[0]['ner_chunk'], full_light_result[0][vocab]):
            
        begin.append(chunk.begin)
        end.append(chunk.end)
        chunks.append(chunk.result)
        codes.append(code.result) 
        all_codes.append(code.metadata['all_k_results'].split(':::'))
        resolutions.append(code.metadata['all_k_resolutions'].split(':::'))
        all_distances.append(code.metadata['all_k_distances'].split(':::'))
        all_cosines.append(code.metadata['all_k_cosine_distances'].split(':::'))
        
    df = pd.DataFrame({'chunks':chunks, 'begin': begin, 'end':end, 'code':codes,'all_codes':all_codes, 
                       'resolutions':resolutions, 'all_distances':all_cosines})
        
    return df



## Test the New Model
We will create a pipeline that contains both of the models and check the results by using `LightPipeline`.

In [22]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("ner_chunk")

sbert_embedder = BertSentenceEmbeddings.pretrained('sbiobert_base_cased_mli', 'en','clinical/models')\
      .setInputCols(["ner_chunk"])\
      .setOutputCol("sbert_embeddings")\
      .setCaseSensitive(False)

first_model = SentenceEntityResolverModel.load("sbiobertresolve_snomed_model") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("first_code")

second_model = SentenceEntityResolverModel.load("new_sbiobertresolve_snomed_model") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("second_code")


pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        first_model,
        second_model])

snomed_lp = LightPipeline(pipelineModel)


sbiobert_base_cased_mli download started this may take some time.
Approximate size to download 384.3 MB
[OK!]


Lets test our models with the terms that we added into the new dataset and the ones that comes from the main model (`Stomach ache` and `Pins and needles`).

In [23]:
chunk_list = ["Dislocation New Term", "Arthrotec", "Raised serum creatinine", "New Drug", "athlete's foot", "Stomach ache", "Pins and needles"]
chunk_list

['Dislocation New Term',
 'Arthrotec',
 'Raised serum creatinine',
 'New Drug',
 "athlete's foot",
 'Stomach ache',
 'Pins and needles']

In [24]:
from IPython.display import display

for chunk in chunk_list:

    print ('\n >>','/'*30, chunk, '/'*30, '\n')
    
    print('First Model Result:')
    display(get_codes (snomed_lp, chunk, vocab='first_code'))
    
    print('\n Second Model Result:')
    display(get_codes (snomed_lp, chunk, vocab='second_code'))


 >> ////////////////////////////// Dislocation New Term ////////////////////////////// 

First Model Result:


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dislocation New Term,0,19,2764000,"[2764000, 125667009, 429513001, 415749005, 202855006, 414469009, 249966004, 14351000168102, 12441001, 4308002, 21288011000036105, 54981004, 698065002, 1806006, 34840004, 283902008, 419076005, 53523011000036100, 3530011000036104, 23056005]","[Joint crepitus, Contusion, Rupture of Achilles tendon, Rupture of tendon, Lateral epicondylitis, Impending shock, Spasmodic movement, Seroquel, Epistaxis, Repetitive strain injury, diclofenac, Charleyhorse, Acid reflux, Eruption, Tendonitis, Has delayed recall, Allergic reaction, Diovan, Lopid, Sciatica]","[0.2654, 0.2695, 0.3075, 0.3011, 0.3281, 0.3136, 0.3238, 0.3381, 0.3422, 0.3231, 0.3598, 0.3562, 0.3352, 0.3335, 0.3447, 0.3418, 0.3383, 0.3491, 0.3609, 0.3472]"



 Second Model Result:


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dislocation New Term,0,19,XXXX67008,"[XXXX67008, XXXXXXXX00036101, 2764000, 125667009, 429513001, 415749005, 202855006, 414469009, 249966004, 14351000168102, 12441001, 4308002, 21288011000036105, 54981004, 698065002, 1806006, 34840004, 283902008, 419076005, 53523011000036100]","[Dislocation New Term, New Drug, Joint crepitus, Contusion, Rupture of Achilles tendon, Rupture of tendon, Lateral epicondylitis, Impending shock, Spasmodic movement, Seroquel, Epistaxis, Repetitive strain injury, diclofenac, Charleyhorse, Acid reflux, Eruption, Tendonitis, Has delayed recall, Allergic reaction, Diovan]","[0.0000, 0.2513, 0.2654, 0.2695, 0.3075, 0.3011, 0.3281, 0.3136, 0.3238, 0.3381, 0.3422, 0.3231, 0.3598, 0.3562, 0.3352, 0.3335, 0.3447, 0.3418, 0.3383, 0.3491]"



 >> ////////////////////////////// Arthrotec ////////////////////////////// 

First Model Result:


...

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Stomach ache,0,11,271681002,"[271681002, 162059005, 116289008, 16331000, 119416008, 21522001, 162043005, 82991003, 248490000, 36349006, 25064002]","[Stomach ache, Upset stomach, Abdominal bloating, Heartburn, Epigastric discomfort, Abdominal pain, Hunger pain, Generalised aches and pains, Bloating symptom, Burning pain, Headache]","[0.0000, 0.0699, 0.0826, 0.0854, 0.0902, 0.0915, 0.0919, 0.1028, 0.1087, 0.1092, 0.1102]"



 >> ////////////////////////////// Pins and needles ////////////////////////////// 

First Model Result:


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Pins and needles,0,15,62507009,"[62507009, 37567005, 17971005, 247472004, 131148009, 91019004, 43498006, 417237009, 283050005, 161891005, 271807003]","[Pins and needles, Acenaesthesia, Sedated, Hives, Bleeding, Paraesthesia, Body fluid retention, Blister, Abrasion of eye region, Backache, Rash]","[0.0000, 0.2716, 0.2708, 0.2917, 0.2857, 0.2918, 0.2834, 0.3023, 0.2994, 0.2994, 0.2969]"



 Second Model Result:


,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Pins and needles,0,15,62507009,"[62507009, 37567005, 17971005, 247472004, 131148009, 91019004, 43498006, 417237009, 283050005, 161891005, 271807003]","[Pins and needles, Acenaesthesia, Sedated, Hives, Bleeding, Paraesthesia, Body fluid retention, Blister, Abrasion of eye region, Backache, Rash]","[0.0000, 0.2716, 0.2708, 0.2917, 0.2857, 0.2918, 0.2834, 0.3023, 0.2994, 0.2994, 0.2969]"


### Conclusion
As you can see in the results;

- The resolutions of the new added terms are at the top of the results.

- The terms that we changed the concept_codes, are at the top of the resuls.

- The close terms results are resolved successfully.

- The terms that comes from the main dataset are resolved with the same result.

## Overriding Codes

We can override new codes over the existing codes if in pretrained Sentence Entity Resolver Model by using `.setOverrideExistingCodes(True)` . For example, you want to add a new term to a pretrained resolver model, and if the code of term already exists in the pretrained model, when you .setOverwriteExistingCode(True), it removes all the same codes and their descriptions from the model, then you will have just the new term with its code in the fine-tuned model.

In [25]:
snomed_data.show()

+-----------------+--------------------+--------------------+--------------------+--------------------+
|        conceptId|               _term|                term|           ner_chunk|     bert_embeddings|
+-----------------+--------------------+--------------------+--------------------+--------------------+
|        161891005|            Backache|            backache|[{document, 0, 7,...|[{sentence_embedd...|
|        418290006|             Itching|               itchy|[{document, 0, 6,...|[{sentence_embedd...|
|         35489007|          Depression|very serious depr...|[{document, 0, 9,...|[{sentence_embedd...|
|         10601006|  Pain in lower limb|        pain in legs|[{document, 0, 17...|[{sentence_embedd...|
|        386806002|  Impaired cognition|  Impaired cognition|[{document, 0, 17...|[{sentence_embedd...|
|        386807006|   Memory impairment| memory difficulties|[{document, 0, 16...|[{sentence_embedd...|
|         21499005|Feeling agitated ...|   Severe aggitation|[{d

In [26]:
display(get_codes (snomed_lp, "Backache", vocab='first_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Backache,0,7,161891005,"[161891005, 279039007, 48926009, 404640003, 367391008, 16269008, 116289008, 81680005, 25064002]","[Backache, Low back pain, Pain in spine, Dizziness, Malaise, Neuralgia, Abdominal bloating, Neck pain, Headache]","[0.0000, 0.1411, 0.1410, 0.1426, 0.1517, 0.1569, 0.1520, 0.1530, 0.1571]"


In [27]:
display(get_codes (snomed_lp, "toothache", vocab='first_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,toothache,0,8,288939007,"[288939007, 404640003, 22253000, 161891005, 25064002]","[Difficulty swallowing, Dizziness, Pain, Backache, Headache]","[0.1266, 0.1317, 0.1340, 0.1390, 0.1360]"


Lets add a new term `chestache`has the same code with `Bachache`, `161891005` and `.setOverrideExistingCodes(True)`.

In [28]:
override_data = spark.createDataFrame(pd.DataFrame({"conceptId":["161891005"], "_term": ["toothache"], "term": ["toothache"]}))
override_data.show()

+---------+---------+---------+
|conceptId|    _term|     term|
+---------+---------+---------+
|161891005|toothache|toothache|
+---------+---------+---------+



In [29]:
override_data = snomed_emb_model.transform(override_data)
override_data.show()

+---------+---------+---------+--------------------+--------------------+
|conceptId|    _term|     term|           ner_chunk|     bert_embeddings|
+---------+---------+---------+--------------------+--------------------+
|161891005|toothache|toothache|[{document, 0, 8,...|[{sentence_embedd...|
+---------+---------+---------+--------------------+--------------------+



In [30]:
overrided_model = bertExtractor.setPretrainedModelPath("/content/sbiobertresolve_snomed_model").setOverrideExistingCodes(True).fit(override_data)
overrided_model.write().overwrite().save("overrided_model")

In [31]:
overrided_resolver = SentenceEntityResolverModel.load("overrided_model") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("overrided_code")


overrided_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        first_model,
        overrided_resolver])

overrided_lp = LightPipeline(overrided_pipelineModel)

In [32]:
# original model

display(get_codes (overrided_lp, "Backache", vocab='first_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Backache,0,7,161891005,"[161891005, 279039007, 48926009, 404640003, 367391008, 16269008, 116289008, 81680005, 25064002]","[Backache, Low back pain, Pain in spine, Dizziness, Malaise, Neuralgia, Abdominal bloating, Neck pain, Headache]","[0.0000, 0.1411, 0.1410, 0.1426, 0.1517, 0.1569, 0.1520, 0.1530, 0.1571]"


In [33]:
# overrided model

display(get_codes (overrided_lp, "Backache", vocab='overrided_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Backache,0,7,161891005,"[161891005, 279039007, 48926009, 404640003, 367391008, 16269008, 116289008, 81680005, 25064002]","[toothache, Low back pain, Pain in spine, Dizziness, Malaise, Neuralgia, Abdominal bloating, Neck pain, Headache]","[0.1390, 0.1411, 0.1410, 0.1426, 0.1517, 0.1569, 0.1520, 0.1530, 0.1571]"


In [34]:
# overrided model

display(get_codes (overrided_lp, "toothache", vocab='overrided_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,toothache,0,8,161891005,"[161891005, 288939007, 404640003, 22253000, 25064002, 367391008, 53057004, 45534005]","[toothache, Difficulty swallowing, Dizziness, Pain, Headache, Malaise, Hand pain, Glossitis]","[0.0000, 0.1266, 0.1317, 0.1340, 0.1360, 0.1415, 0.1379, 0.1457]"


As you can see, there is no more `Backache` -> `161891005`. Now all descriptions of `161891005` code are removed and there is just one description `tootache` -> `161891005` for this code.

## Drop List Of Codes

We can drop codes from a pretrained resolver model bu using .`setDropCodesList`. Lets remove `161891005` (toothache) and `404640003` (Dizzeness) from the overrided model.

In [35]:
blackListedModel = bertExtractor.\
    setPretrainedModelPath("overrided_model").\
    setNormalizedCol("_term").\
    setDropCodesList(["161891005", "404640003"]).\
    fit(override_data.limit(0))

In [36]:
blackListedModel.write().overwrite().save("blackListedModel")

In [37]:
blackListed_resolver = SentenceEntityResolverModel.load("blackListedModel") \
      .setInputCols(["ner_chunk", "sbert_embeddings"]) \
      .setOutputCol("blackListed_code")


blackListed_pipelineModel = PipelineModel(
    stages = [
        documentAssembler,
        sbert_embedder,
        overrided_resolver,
        blackListed_resolver])

blackListed_lp = LightPipeline(blackListed_pipelineModel)

In [38]:
# overrided model

display(get_codes (blackListed_lp, "toothache", vocab='overrided_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,toothache,0,8,161891005,"[161891005, 288939007, 404640003, 22253000, 25064002, 367391008, 53057004, 45534005]","[toothache, Difficulty swallowing, Dizziness, Pain, Headache, Malaise, Hand pain, Glossitis]","[0.0000, 0.1266, 0.1317, 0.1340, 0.1360, 0.1415, 0.1379, 0.1457]"


In [39]:
# blackListed model

display(get_codes (blackListed_lp, "toothache", vocab='blackListed_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,toothache,0,8,288939007,"[288939007, 22253000, 25064002, 367391008, 53057004, 45534005, 248490000, 18876004]","[Difficulty swallowing, Pain, Headache, Malaise, Hand pain, Glossitis, Bloating symptom, Pain in finger]","[0.1266, 0.1340, 0.1360, 0.1415, 0.1379, 0.1457, 0.1395, 0.1412]"


In [40]:
# overrided model

display(get_codes (blackListed_lp, "Dizziness", vocab='overrided_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dizziness,0,8,404640003,"[404640003, 271782001, 119416008, 271713000, 248490000, 373931001, 310501001, 309838005, 367391008, 301026000, 214264003, 55929007, 7006003, 399153001]","[Dizziness, Drowsy, Epigastric discomfort, General unsteadiness, Bloating symptom, Sensation of heaviness in limbs, Numbness of limbs, Emotional upset, Malaise, Loss of confidence, Lethargy, Feeling irritable, Clumsiness, Vertigo]","[0.0000, 0.0811, 0.0838, 0.0858, 0.0877, 0.0893, 0.0929, 0.0979, 0.1055, 0.1028, 0.1055, 0.1024, 0.1063, 0.1113]"


In [41]:
# blackListed model

display(get_codes (blackListed_lp, "Dizziness", vocab='blackListed_code'))

,chunks,begin,end,code,all_codes,resolutions,all_distances
0,Dizziness,0,8,271782001,"[271782001, 119416008, 271713000, 248490000, 373931001, 310501001, 309838005, 367391008, 301026000, 214264003, 55929007, 7006003, 399153001, 271795006, 101000119102, 53057004]","[Drowsy, Epigastric discomfort, General unsteadiness, Bloating symptom, Sensation of heaviness in limbs, Numbness of limbs, Emotional upset, Malaise, Loss of confidence, Lethargy, Feeling irritable, Clumsiness, Vertigo, Malaise and fatigue, Numbness and tingling sensation of skin, Hand pain]","[0.0811, 0.0838, 0.0858, 0.0877, 0.0893, 0.0929, 0.0979, 0.1055, 0.1028, 0.1055, 0.1024, 0.1063, 0.1113, 0.1097, 0.1104, 0.1125]"


As you can see, `161891005`, `404640003` codes are removed drom the model.